###if doesn't work try changing the cookie. You can get a cookie from the klook website. 

In [10]:
import requests

def fetch_hotel_reviews(hotel_id, size=25, start=1, review_tag_id="1001"):
    url = "https://www.klook.com/v1/hotelapiserv/hotelapi/review/list"
    
    cookie = 'kepler_id=48a81c06-3467-4b22-824f-f6eb53df5572; klk_currency=USD; klk_rdc=US; klk_ps=1; __lt__cid=951173ca-0dd2-4191-ad46-452a629d2fc9; __lt__cid.c83939be=951173ca-0dd2-4191-ad46-452a629d2fc9; __lt__sid=48ce62e6-06ce0eda; __lt__sid.c83939be=48ce62e6-06ce0eda; _fwb=16V28epZ3XAq2wOk0kbSi7.1720947634400; _yjsu_yjad=1720947634.9e0c68e0-09d9-4b40-81f4-52934c45fe6a; _tt_enable_cookie=1; _ttp=uEcKI2sTZauRwXO8gIPz3XCPQ7d; KSID=DS.1395eb3031eb471a6756ffd98f59679b; dable_uid=60184520.1720947635136; _gcl_au=1.1.2070249969.1720947635; _gid=GA1.2.1369394506.1720947635; JSESSIONID=693B6AC9129B9C223E4B137F1A7495DF; KOUNT_SESSION_ID=693B6AC9129B9C223E4B137F1A7495DF; clientside-cookie=a76b43e7173fbdf53c020538cb0c85cc2d6d9a9d36d1b6f553d96b80c89d0cb82ee97fd66797e990f07367b9b0e33b83ca90958c400d772074d634440b889188b0a0c006abf8d3b2f786bb5544c7ace85a210896f12303cc5358c870e80b18105034aa6cab23ba188e1963b2cc36c913ffc05b90156164d79cd5f4a278d443a42c54db801d30ffe5aa4b278a1d33416c654b126fbd66f311932d06; klk_lang=en; klk_ga_sn=5962843128..1720948599261; wcs_bt=s_2cb388a4aa34:1720948599; _uetsid=87d5236041bf11efa9b957caca3cc7e1; _uetvid=87d5255041bf11efaeed4f2e0bec157b; _dc_gtm_UA-86696233-1=1; _ga_FW3CMDM313=GS1.1.1720947634.1.1.1720948599.0.0.0; _ga_V8S4KC8ZXR=GS1.1.1720947634.1.1.1720948600.4.0.0; datadome=3z9~EyNRJ7OZltZgR~vdtGKpaEbNvbrO9UjYEQ8vuM1plXL6J5evdkXk7P5zpNF83rnDCXWwiwlMwxarBlVxKfN~~K1KUiKUIkKirpjf~yn_FdAhnGoVuN0gCo~7dYAO; _ga_TH9DNLM4ST=GS1.1.1720947635.1.1.1720948601.5.0.0; forterToken=c1fc0a8c4e954d6d9e107ecfa0bbdfb0_1720948600206__UDF43-m4_21ck_; _ga=GA1.2.1368613852.1720947635; klk_i_sn=4323870682..1720948615321'

    payload = {
        "hotel_id": hotel_id,
        "size": size,
        "start": start,
        "review_filter": {
            "review_tag_list": [
                {"id": review_tag_id}
            ]
        },
        "pin_id": ""
    }
    
    headers = {
        'cookie': cookie,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
        'content-type': 'application/json;charset=UTF-8'
    }
    
    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()




In [11]:
import csv
import os
from datetime import datetime

def check_and_write_headers(csv_file, headers):
    """
    Check if the CSV file exists and write headers if it doesn't.
    """
    file_exists = os.path.isfile(csv_file)
    if not file_exists:
        with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(headers)

def filter_reviews(review_list):
    """
    Filter reviews to include only English reviews from 2022 to 2024.
    """
    filtered_reviews = []

    for review in review_list:
        review_detail = review.get('review_detail', {})

        # Only include English reviews
        if not review_detail.get('language', '').startswith('en'):
            continue

        review_time = review_detail.get('review_time')

        # Check if the review time is between 2022 and 2024
        try:
            review_date = datetime.strptime(review_time, "%Y-%m-%d")
            if not (review_date.year >= 2022 and review_date.year <= 2024):
                continue
            
            filtered_reviews.append(review)
        except ValueError:
            # Handle the case where the date format is incorrect
            continue
    
    return filtered_reviews

def write_reviews_to_csv(csv_file, review_list):
    """
    Write review data to the CSV file.
    """
    row_count = 0

    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        for review in review_list:
            review_detail = review.get('review_detail', {})
            review_time = review_detail.get('review_time')
            review_id = review.get('review_id')
            review_content = review_detail.get('review_content')
            review_score = review_detail.get('review_score', {}).get('avg_score')

            # Write the row to the CSV file
            writer.writerow([review_id, review_content, review_score, review_time])
            
            # Increment the counter
            row_count += 1

    return row_count



In [12]:
import pandas as pd
def main():
    # Define the headers for the CSV file
    headers = ["Review ID", "Review Content", "Review Score", "Review Time"]
    
    # Directory to save CSV files
    output_dir = "klook_hotel_reviews"
    os.makedirs(output_dir, exist_ok=True)

    # List of hotels with names and IDs
    hotels = [
        {"name": "Waterfront Hotel and Casino", "id": "119354"},
        {"name": "Seda Ayala Center Cebu", "id": "449272"},
        {"name": "bai Hotel", "id": "253867"},
        {"name": "Fairfield by Marriott Cebu", "id": "1285962"},
        {"name": "Jpark Island Resort and Waterpark", "id": "449304"},
        {"name": "Dusit Thani Mactan", "id": "280179"}
    ]
      
    size = 100

    for hotel in hotels:
        hotel_name = hotel["name"]
        hotel_id = hotel["id"]
        start = 1
        total_reviews_written = 0
        csv_file = os.path.join(output_dir, f"{hotel_name.replace(' ', '_').lower()}_reviews_2022_2024.csv")
        
        check_and_write_headers(csv_file, headers)
        
        print(f"scraping {hotel_name}")
        while True:
            print(f"iteration: {start}")
            # Fetch reviews
            response_data = fetch_hotel_reviews(hotel_id, size=size, start=start)
            review_list = response_data.get('result', {}).get('review_list', [])
            
            if not review_list:
                break
            
            # Filter reviews
            filtered_reviews = filter_reviews(review_list)
            
            # Write reviews to CSV and get the count of rows written
            row_count = write_reviews_to_csv(csv_file, filtered_reviews)
            
            total_reviews_written += row_count
            start += 1
        
        print(f"English reviews from 2022 to 2024 for hotel {hotel_name} have been written to {csv_file}")
        print(f"Total number of rows written for hotel {hotel_name}: {total_reviews_written}")

if __name__ == "__main__":
    main()

scraping Waterfront Hotel and Casino
iteration: 1
iteration: 2
English reviews from 2022 to 2024 for hotel Waterfront Hotel and Casino have been written to klook_hotel_reviews\waterfront_hotel_and_casino_reviews_2022_2024.csv
Total number of rows written for hotel Waterfront Hotel and Casino: 37
scraping Seda Ayala Center Cebu
iteration: 1
iteration: 2
English reviews from 2022 to 2024 for hotel Seda Ayala Center Cebu have been written to klook_hotel_reviews\seda_ayala_center_cebu_reviews_2022_2024.csv
Total number of rows written for hotel Seda Ayala Center Cebu: 36
scraping bai Hotel
iteration: 1
iteration: 2
iteration: 3
English reviews from 2022 to 2024 for hotel bai Hotel have been written to klook_hotel_reviews\bai_hotel_reviews_2022_2024.csv
Total number of rows written for hotel bai Hotel: 114
scraping Fairfield by Marriott Cebu
iteration: 1
iteration: 2
English reviews from 2022 to 2024 for hotel Fairfield by Marriott Cebu have been written to klook_hotel_reviews\fairfield_by_